In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, reset_output
from bokeh.embed import file_html
from bokeh.models import LabelSet, Label, ColumnDataSource, FactorRange
from bokeh.transform import factor_cmap
import bokeh.plotting as bp
%matplotlib inline

In [8]:
data = pd.read_csv("./MC1/mc1-reports-data.csv",float_precision='high',parse_dates=True,
                   infer_datetime_format=True,index_col=0)
data = data.sort_index()
data

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
time,,,,,,,
2020-04-06 00:00:00,2.0,4.0,1.0,NaN,4.0,NaN,17
2020-04-06 00:00:00,2.0,8.0,5.0,NaN,3.0,0.0,2
2020-04-06 00:00:00,9.0,7.0,5.0,NaN,4.0,NaN,8
2020-04-06 00:00:00,10.0,2.0,9.0,NaN,10.0,NaN,4
2020-04-06 00:00:00,6.0,2.0,9.0,NaN,4.0,1.0,15
2020-04-06 00:00:00,9.0,7.0,4.0,NaN,8.0,0.0,18
2020-04-06 00:00:00,8.0,1.0,10.0,NaN,3.0,1.0,4
2020-04-06 00:00:00,6.0,3.0,10.0,10.0,9.0,NaN,3
2020-04-06 00:05:00,10.0,3.0,8.0,4.0,1.0,NaN,6


In [10]:
data = data.fillna(0)

In [ ]:
1,3,5,6,9,11,16

In [11]:
data.groupby("location").mean()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity
location,,,,,,
1,4.740072,4.759326,4.867629,4.974729,4.904934,0.329723
2,3.413544,4.313544,4.779777,0.089054,1.960111,1.266605
3,7.275434,8.461396,7.267085,7.055264,5.831991,4.590100
4,5.468654,4.224049,4.213772,0.096608,3.692018,4.263789
5,4.205263,5.065497,4.083626,2.110526,3.494737,0.519298
6,2.937975,3.062975,3.318987,3.306013,3.516139,0.766139
7,0.098266,7.838150,5.919075,0.034682,0.098266,4.421965
8,7.499748,7.457772,7.658291,0.087551,5.781986,1.527252
9,7.456003,5.726353,6.591291,5.184970,6.168486,0.973511


In [13]:
reset_output() #Para que no abra un nuevo navegador
output_notebook()
#output_file('./resultados/image10.html')
TITLE = "title"
TOOLS = "hover,box_select,pan,wheel_zoom,box_zoom,reset,save"

p = figure(tools=TOOLS, toolbar_location="above",width=900, height=500,
           x_range=(30,180), y_range=(35,185), title=TITLE)
#MAPA
p.image_url(url=['MC1/mapa.svg'], x=50,y=180,w=100,h=150)
p.hover.tooltips = [
    ("name", "@name"),
    ("neighborhood", "@location"),
    ("intensity:", "@shake_intensity")
]
show(p)

Loading BokehJS ...